In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('/Users/brianoktavec/MSC550/Assignment_7/HousePrices.csv')

In [3]:
df.head()

,Price,SqFt,Bed,Bath,Offers
0,114300,1790,2,2,2.0
1,114200,2030,4,2,3.0
2,114800,1740,3,2,1.0
3,94700,1980,3,2,NaN
4,119800,2130,3,3,3.0


# #1

In [10]:
df_clean = df.dropna()

In [11]:
df_clean.head()

,Price,SqFt,Bed,Bath,Offers
0,114300,1790,2,2,2.0
1,114200,2030,4,2,3.0
2,114800,1740,3,2,1.0
4,119800,2130,3,3,3.0
5,114600,1780,3,2,2.0


In [14]:
X = df_clean[['SqFt', 'Bed', 'Bath', 'Offers']]
X = pd.get_dummies(X, drop_first=True)
y = df_clean['Price']


import statsmodels.api as sm
ols = sm.OLS(y, X)
lm = ols.fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:                  Price   R-squared (uncentered):                   0.989
Model:                            OLS   Adj. R-squared (uncentered):              0.988
Method:                 Least Squares   F-statistic:                              2302.
Date:                Sat, 01 Apr 2023   Prob (F-statistic):                   8.91e-103
Time:                        11:23:38   Log-Likelihood:                         -1220.5
No. Observations:                 111   AIC:                                      2449.
Df Residuals:                     107   BIC:                                      2460.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

# #2

In [15]:
df['Offers'].fillna(df['Offers'].mean(), inplace=True)

In [19]:
ols = sm.OLS(df['Price'], sm.add_constant(df.iloc[:, 1:5]))
lm1 = ols.fit()
print(lm1.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.636
Model:                            OLS   Adj. R-squared:                  0.625
Method:                 Least Squares   F-statistic:                     53.84
Date:                Sat, 01 Apr 2023   Prob (F-statistic):           3.76e-26
Time:                        11:30:03   Log-Likelihood:                -1421.8
No. Observations:                 128   AIC:                             2854.
Df Residuals:                     123   BIC:                             2868.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.175e+04   1.39e+04     -0.844      0.4

# #3

In [20]:
complete_cases = df.dropna()

In [22]:
X = complete_cases[["SqFt", "Bed", "Bath"]]
y = complete_cases["Offers"]
model = sm.OLS(y, sm.add_constant(X)).fit()

In [23]:
incomplete_cases = df[df["Offers"].isnull()]
X_missing = incomplete_cases[["SqFt", "Bed", "Bath"]]
y_missing = model.predict(sm.add_constant(X_missing))
df.loc[df["Offers"].isnull(), "Offers"] = y_missing

In [24]:
ols = sm.OLS(df.Price, sm.add_constant(df.iloc[:, 1:5]))
lm2 = ols.fit()
print(lm2.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.636
Model:                            OLS   Adj. R-squared:                  0.625
Method:                 Least Squares   F-statistic:                     53.84
Date:                Sat, 01 Apr 2023   Prob (F-statistic):           3.76e-26
Time:                        11:32:14   Log-Likelihood:                -1421.8
No. Observations:                 128   AIC:                             2854.
Df Residuals:                     123   BIC:                             2868.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.175e+04   1.39e+04     -0.844      0.4

# #4

In [36]:
df_test = df[df['Offers'].isnull()==True]
df_train = df[df['Offers'].isnull()==False]

ols = sm.OLS(df_train[["Offers"]], sm.add_constant(df_train[["SqFt","Bed","Bath"]]))
lm3 = ols.fit()

mean = lm3.predict(sm.add_constant(df_test[["SqFt","Bed","Bath"]]))
std = lm3.resid.std()
rand_pred = np.random.normal(size = df_test.shape[0], loc = mean, scale = std)
df_test[["Offers"]] = rand_pred
df1 = pd.concat([df_train,df_test])

<ipython-input-36-21bfad7fef86>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[["Offers"]] = rand_pred


In [38]:
ols = sm.OLS(df1['Price'], sm.add_constant(df1.iloc[:, 1:5]))
lm4 = ols.fit()

print(lm4.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.636
Model:                            OLS   Adj. R-squared:                  0.625
Method:                 Least Squares   F-statistic:                     53.84
Date:                Sat, 01 Apr 2023   Prob (F-statistic):           3.76e-26
Time:                        20:38:38   Log-Likelihood:                -1421.8
No. Observations:                 128   AIC:                             2854.
Df Residuals:                     123   BIC:                             2868.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.175e+04   1.39e+04     -0.844      0.4

# #5

In [40]:
print("P-values for Model 1")
print(lm.pvalues)

P-values for Model 1
const    0.743504
SqFt     0.001102
Bed      0.000472
Bath     0.001685
dtype: float64


In [41]:
print("P-values for Model 2")
print(lm1.pvalues)

P-values for Model 2
const     4.004826e-01
SqFt      1.350343e-08
Bed       3.291760e-04
Bath      4.666256e-06
Offers    3.265925e-13
dtype: float64


In [42]:
print("P-values for Model 3")
print(lm2.pvalues)

P-values for Model 3
const     4.004826e-01
SqFt      1.350343e-08
Bed       3.291760e-04
Bath      4.666256e-06
Offers    3.265925e-13
dtype: float64


In [43]:
print("P-values for Model 4")
print(lm4.pvalues)

P-values for Model 4
const     4.004826e-01
SqFt      1.350343e-08
Bed       3.291760e-04
Bath      4.666256e-06
Offers    3.265925e-13
dtype: float64


# #6

In [50]:
df_test = df[df['Offers'].isnull()==True]
df_train = df[df['Offers'].isnull()==False]

ols = sm.OLS(df_train[["Offers"]], sm.add_constant(df_train[["SqFt", "Bed", "Bath"]]))
lm = ols.fit()

mean = lm.predict(sm.add_constant(df_test[["SqFt", "Bed", "Bath"]]))
std = lm.resid.std()

model = []
coef = np.zeros((4,5))
for i in range(0,5):
    rand_pred = np.random.normal(size = df_test.shape[0], loc = mean, scale = std)
    df_test[["Offers"]] = rand_pred
    df1 = pd.concat([df_train,df_test])
    lm = sm.OLS(df1[["Price"]], sm.add_constant(df1[["SqFt", "Bed", "Bath", "Offers"]])).fit()
    model.append(lm)
    coef[:,i] = lm.params[1:]

coef.mean(axis=1)


<ipython-input-50-049405eee026>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[["Offers"]] = rand_pred


array([    54.32015702,   8737.16775803,  16434.79409602, -13716.27629802])

In [56]:
integrated_coefficients = coef.mean(axis=1)
integrated_model = sm.OLS(df1[["Price"]], sm.add_constant(df1[["SqFt","Bed","Bath","Offers"]])).fit()

In [57]:
print(integrated_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.636
Model:                            OLS   Adj. R-squared:                  0.625
Method:                 Least Squares   F-statistic:                     53.84
Date:                Sat, 01 Apr 2023   Prob (F-statistic):           3.76e-26
Time:                        21:22:12   Log-Likelihood:                -1421.8
No. Observations:                 128   AIC:                             2854.
Df Residuals:                     123   BIC:                             2868.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.175e+04   1.39e+04     -0.844      0.4